In [ ]:

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
import pandas as pd

# Your data: numpy array of shape (n_samples, n_features)
X = train_data.squeeze()  # Adjust if 3D is (n_samples, n_timestamps, 1)

# Define grid of hyperparameters
param_grid = {
    'n_clusters': [2, 3, 4, 5, 6, 7, 8],
    'init': ['k-means++', 'random'],
    'n_init': [10, 20],
    'max_iter': [300, 500]
}

import itertools
results = []

# Cartesian product of parameters
for n_clusters, init, n_init, max_iter in itertools.product(
    param_grid['n_clusters'],
    param_grid['init'],
    param_grid['n_init'],
    param_grid['max_iter']
):
    kmeans = KMeans(
        n_clusters=n_clusters,
        init=init,
        n_init=n_init,
        max_iter=max_iter,
        random_state=42
    )
    labels = kmeans.fit_predict(X)
    # Calculate scores only if more than 1 cluster
    if len(set(labels)) > 1:
        sil_score = silhouette_score(X, labels)
        db_score = davies_bouldin_score(X, labels)
        results.append({
            'n_clusters': n_clusters,
            'init': init,
            'n_init': n_init,
            'max_iter': max_iter,
            'silhouette_score': sil_score,
            'db_score': db_score
        })

# Results as a DataFrame, sorted by silhouette score (descending)
results_df = pd.DataFrame(results).sort_values(by='silhouette_score', ascending=False)
print(results_df)